# Convolutional Neural Networks in ``gluon``

Now let's see how succinctly we can express a convolutional neural network using ``gluon``. You might be relieved to find out that this too requires hardly any more code than a logistic regression. 

In [12]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
import numpy as np
mx.random.seed(1)

## Set the context

In [13]:
ctx = mx.gpu()

## Grab the MNIST dataset

In [14]:
mnist = mx.test_utils.get_mnist()
batch_size = 64
num_inputs = 784
num_outputs = 10
train_data = mx.io.NDArrayIter(mnist["train_data"], mnist["train_label"], batch_size, shuffle=True)
test_data = mx.io.NDArrayIter(mnist["test_data"], mnist["test_label"], batch_size, shuffle=True)

## Define a convolutional neural network

Again, a few lines here is all we need in order to change the model. Let's add a couple of convolutional layers using ``gluon.nn``.

In [15]:
num_fc = 512
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Conv2D(channels=20, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))            
    net.add(gluon.nn.Conv2D(channels=50, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))            
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(num_fc, activation="relu"))
    net.add(gluon.nn.Dense(num_outputs))

## Parameter initialization


In [16]:
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

## Softmax cross-entropy Loss

In [17]:
loss = gluon.loss.SoftmaxCrossEntropyLoss()

## Optimizer

In [18]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})

## Write evaluation loop to calculate accuracy

In [19]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    data_iterator.reset()
    for i, batch in enumerate(data_iterator):
        data = batch.data[0].as_in_context(ctx)
        label = batch.label[0].as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

## Training Loop

In [20]:
epochs = 10

for e in range(epochs):
    train_data.reset()
    moving_loss = 0.
    for i, batch in enumerate(train_data):
        data = batch.data[0].as_in_context(ctx)
        label = batch.label[0].as_in_context(ctx)
        with autograd.record():
            output = net(data)
            cross_entropy = loss(output, label)
        cross_entropy.backward()
        trainer.step(data.shape[0])
        
        moving_loss = .99 * moving_loss + .01 * nd.mean(cross_entropy).asscalar()
            
    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, moving_loss, train_accuracy, test_accuracy))    

Epoch 0. Loss: 0.0872391404979, Train_acc 0.97921108742, Test_acc 0.979498407643
Epoch 1. Loss: 0.0529439197961, Train_acc 0.987939765458, Test_acc 0.987460191083
Epoch 2. Loss: 0.0381611640977, Train_acc 0.991271321962, Test_acc 0.988554936306
Epoch 3. Loss: 0.0289579770572, Train_acc 0.993336886994, Test_acc 0.989948248408
Epoch 4. Loss: 0.0219234052425, Train_acc 0.994203091684, Test_acc 0.989948248408
Epoch 5. Loss: 0.0175043509644, Train_acc 0.995535714286, Test_acc 0.990445859873
Epoch 6. Loss: 0.0137102735181, Train_acc 0.996118736674, Test_acc 0.990843949045
Epoch 7. Loss: 0.0107276559719, Train_acc 0.99688499467, Test_acc 0.991242038217
Epoch 8. Loss: 0.00918125792343, Train_acc 0.997318097015, Test_acc 0.99034633758
Epoch 9. Loss: 0.0076400288069, Train_acc 0.998101012793, Test_acc 0.991242038217


## Conclusion

You might notice that by using ``gluon``, we get code that runs much faster whether on CPU or GPU. That's largely because ``gluon`` can call down to highly optimized layers that have been written in C++. 

For whinges or inquiries, [open an issue on  GitHub.](https://github.com/zackchase/mxnet-the-straight-dope)